In [1]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
fried_rice_train_dir = '../data/food11/train/fried_rice'
fried_rice_test_dir = '../data/food11/test/fried_rice'

# Collect all image paths in the pizza folders
image_paths = [os.path.join(fried_rice_train_dir, fname) for fname in os.listdir(fried_rice_train_dir)]
image_paths += [os.path.join(fried_rice_test_dir, fname) for fname in os.listdir(fried_rice_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 999 images


In [2]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 8

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/fried_rice\1004221.jpg assigned to cluster 7
../data/food11/train/fried_rice\1008935.jpg assigned to cluster 2
../data/food11/train/fried_rice\1015700.jpg assigned to cluster 4
../data/food11/train/fried_rice\1019981.jpg assigned to cluster 7
../data/food11/train/fried_rice\1021229.jpg assigned to cluster 3
../data/food11/train/fried_rice\1022515.jpg assigned to cluster 4
../data/food11/train/fried_rice\1023430.jpg assigned to cluster 7
../data/food11/train/fried_rice\1028159.jpg assigned to cluster 4
../data/food11/train/fried_rice\1028939.jpg assigned to cluster 4
../data/food11/train/fried_rice\1042399.jpg assigned to cluster 5
../data/food11/train/fried_rice\1043233.jpg assigned to cluster 0
../data/food11/train/fried_rice\1043851.jpg assigned to cluster 5
../data/food11/train/fried_rice\1047951.jpg assigned to cluster 7
../data/food11/train/fried_rice\1052689.jpg assigned to cluster 1
../data/food11/train/fried_rice\1053345.jpg assigned to cluster 1
../data/fo

In [3]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(fried_rice_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(fried_rice_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(fried_rice_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(fried_rice_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Fine-grained apple_pie classification
fried_rice_train_gen = ImageDataGenerator(rescale=1./255)
fried_rice_test_gen = ImageDataGenerator(rescale=1./255)

fried_rice_train_generator = fried_rice_train_gen.flow_from_directory(
    fried_rice_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

fried_rice_test_generator = fried_rice_test_gen.flow_from_directory(
    fried_rice_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Use the same base model (MobileNet)
base_model_fried_rice = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_fried_rice.layers:
    layer.trainable = False

# Add custom layers for apple_pie classification
x_fried_rice = base_model_fried_rice.output
x_fried_rice = GlobalAveragePooling2D()(x_fried_rice)
x_fried_rice = Dense(512, activation='relu')(x_fried_rice)
fried_rice_predictions = Dense(num_clusters, activation='softmax')(x_fried_rice)

# Create and compile the model
fried_rice_model = Model(inputs=base_model_fried_rice.input, outputs=fried_rice_predictions)
fried_rice_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-grained apple_pie classification model
fried_rice_model.fit(
    fried_rice_train_generator,
    steps_per_epoch=fried_rice_train_generator.samples // 32,
    validation_data=fried_rice_test_generator,
    validation_steps=fried_rice_test_generator.samples // 32,
    epochs=30
)

# Save the model
fried_rice_model.save('fine_grained_fried_rice_classifier.h5')

Found 900 images belonging to 8 classes.
Found 99 images belonging to 8 classes.
Epoch 1/30
28/28 [==============================] - 9s 280ms/step - loss: 1.6954 - accuracy: 0.3641 - val_loss: 1.3756 - val_accuracy: 0.5417
Epoch 2/30
28/28 [==============================] - 7s 254ms/step - loss: 0.9627 - accuracy: 0.7200 - val_loss: 0.9879 - val_accuracy: 0.6875
Epoch 3/30
28/28 [==============================] - 7s 242ms/step - loss: 0.6579 - accuracy: 0.8272 - val_loss: 0.7826 - val_accuracy: 0.7396
Epoch 4/30
28/28 [==============================] - 7s 249ms/step - loss: 0.4892 - accuracy: 0.8940 - val_loss: 0.6825 - val_accuracy: 0.7500
Epoch 5/30
28/28 [==============================] - 7s 251ms/step - loss: 0.3889 - accuracy: 0.9147 - val_loss: 0.6196 - val_accuracy: 0.7917
Epoch 6/30
28/28 [==============================] - 7s 252ms/step - loss: 0.3200 - accuracy: 0.9493 - val_loss: 0.5292 - val_accuracy: 0.8438
Epoch 7/30
28/28 [==============================] - 7s 249ms/step -